**BIBLIOGRAFIA:**

https://stackoverflow.com/questions/24226781/changing-user-agent-in-python-3-for-urrlib-request-urlopen/24226797

https://docs.python.org/3/howto/regex.html

https://www.dataquest.io/blog/web-scraping-tutorial-python/

http://toddhayton.com/2015/01/16/scraping-by-example-json-data/

https://stackoverflow.com/questions/48906246/extract-json-from-html-in-python-beautifulsoup?rq=1

https://www.dataquest.io/blog/web-scraping-beautifulsoup/

https://www.elcorteingles.es/supermercado/


In [1]:
import urllib
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from urllib.parse import urlparse
import datetime
import time
import csv
import numpy as np

In [2]:
class Throttle:
    """Add a delay between downloads to the same domain
    """
    def __init__(self, delay):
        # amount of delay between downloads for each domain
        self.delay = delay
        # timestamp of when a domain was last accessed
        self.domains = {}
        
    def wait(self, url):
        domain = urlparse(url).netloc
        last_accessed = self.domains.get(domain)
        
        if self.delay > 0 and last_accessed is not None:
            sleep_secs = self.delay - (datetime.datetime.now() - 
                                       last_accessed).seconds
            if sleep_secs > 0:
                # domain has been accessed recently
                # so need to sleep
                time.sleep(sleep_secs)
        # update the last accessed time
        self.domains[domain] = datetime.datetime.now()


#Creem funció que obrirà els links
def download(url, user_agent='wswp', num_retries=2):
    print ('Downloading:', url)
    headers = {'User-agent': user_agent}
    request = urllib.request.Request(url,None,headers=headers)
    try:
        throttle.wait(path)
        html = urllib.request.urlopen(request).read().decode('utf-8')
    except urllib.request.URLError as e:
        print ('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # retry 5XX HTTP errors
                return download(url, user_agent, num_retries-1)
    return html

#Creem funció que obtindrà les categoríes del productes (menú superior web)
def lookCategoria(url,userAgent):
    html=download(url,userAgent)
    soup = BeautifulSoup(html,'html.parser')
    valors = soup.find_all('a',class_='top_menu-item')
    links = [pt['href'] for pt in valors]
    noms = [pt.get_text() for pt in valors]
    productes = pd.DataFrame({
        "nom": noms, 
        "link": links
    })
    return productes


#Creem funció que obtindrà els tipus de productes
def lookTipu(url,userAgent,categ):
    html=download(url,userAgent)
    soup = BeautifulSoup(html,'html.parser')
    cong = soup.find_all('a',class_='link _primary _level3 tree-link ')
    cong
    links = [pt['href'] for pt in cong]
    noms = [pt.get_text() for pt in cong]
    
    productes = pd.DataFrame({
        "categoria": categ,
        "nom": noms, 
        "link": links
    })
    return productes
 

#Creem funció que obtindrà els productes i quantes paginacions hi ha (dins d'una categoria/tipus de producte)
def lookProd(path,userAgent,categ,tipu):
    html=download(path,userAgent)
    soup = BeautifulSoup(html,'html.parser')
    prod = soup.find(type="application/ld+json")
    jsonData = json.loads(prod.get_text())    #Convert to JSON Object.
    #print(jsonData['itemListElement'])
    elem = jsonData['itemListElement']
    #elem
    noms = [pt['name'] for pt in elem]
    #noms
    urls = [pt['url'] for pt in elem]
    #págines
    infopag = soup.find(class_='pagination c12 js-pagination')
    if infopag is None:
        numpag = 0
    else:
        numpag = infopag['data-pagination-total']
    #urls
    productes = pd.DataFrame({
        "categoria": categ,
        "tipu": tipu,
        "nom": noms, 
        "link": urls,
        "paginas": int(numpag)
    })
    return productes
    

#Obté els productes (idem que lookProd) però no inclou infromació de paginació
def lookProdSeg(path,userAgent,categ,tipu):
    html=download(path,userAgent)
    soup = BeautifulSoup(html,'html.parser')
    prod = soup.find(type="application/ld+json")
    jsonData = json.loads(prod.get_text())    #Convert to JSON Object.
    #print(jsonData['itemListElement'])
    elem = jsonData['itemListElement']
    #elem
    noms = [pt['name'] for pt in elem]
    #noms
    urls = [pt['url'] for pt in elem]
    #urls
    productes = pd.DataFrame({
        "categoria": categ,
        "tipu": tipu,
        "nom": noms, 
        "link": urls
    })
    return productes

#Creem funció que obtindrà els alergènics de cada producte i la url de la seva imatge.
#Només retornarem aquells productes que tenen ingredients informats al Web.
def lookAlerg(path,userAgent,producte):
    html=download(path,userAgent)
    soup = BeautifulSoup(html,'html.parser')
    info = soup.find_all('h3',class_='info-title')
    ingredients = [pt.get_text() for pt in info]
    productes = []
    #Només busquem al·lèrgens si hi ha ingredients al web
    if 'Ingredientes y alérgenos' in ingredients:
        alerg = soup.find_all('b')
        alergens = [pt.get_text() for pt in alerg]
        #obtenim les urls de les imatges
        infoImg = soup.find_all('div',class_='zoom')
        imatge = infoImg[0].find('img')
        imatge = imatge['data-zoom-src']
        #unifiquem la informació d'alèrgens per evitar ingredients duplicats
        productes = [producte, list (set (alergens)), imatge]

    return productes


#Creem funció que descarrega imatges donada una url
def load_requests(source_url,destination_url):
    r = requests.get(source_url, stream = True)
    if r.status_code == 200:
        aSplit = source_url.split('/')
        ruta = destination_url+aSplit[len(aSplit)-1]
        output = open(ruta,"wb")
        for chunk in r:
            output.write(chunk)
        output.close()


        
#Definim les constants que utilitzarem durant l'execució
path="http://www.elcorteingles.es"
destiCSV="D:/UOC/TipologiaPRAC1/"
filename = "alergens.csv"
destiIMG="D:/UOC/TipologiaPRAC1/imatges/"
UserAg="student-uoc"
delay=0.1

In [3]:
#Comprobem el robots.txt a veure si el nostre userAgent no té cap objecció d'accedir.
#Veiem que la secció de Supermercat no té cap suggeriment de no accés. Per tant procedim amb el nostre codi d'extracció.
#No recorrerem els sitemats suggerits ja que anirem directament a la secció Supermercat que ens interessa.
throttle = Throttle(delay)
throttle.wait(path)

robot=download(path+"/robots.txt",UserAg)
print(robot)

Downloading: http://www.elcorteingles.es/robots.txt
User-agent: *
Disallow: /login
Disallow: */checkout/*
Disallow: /checkout-signin
Disallow: /profile2
Disallow: /*/search
Disallow: /entradas/search/*
Disallow: /entradas/revista/tag/*
Disallow: /ocio/search/*
Disallow: /ocio/newsletters/*
Disallow: /store/*
Disallow: */cart/*
Disallow: /aptc/c/*
Disallow: /compra/*

Disallow: /empresas/isbutpage/*
Disallow: /empresas/busqueda/*
Disallow: /empresas/registrarse/*
Disallow: /empresas/es/informacion/pdf/politica_de_cookies.pdf
Disallow: /empresas/cesta/*
Disallow: /electronica/telefonia/pulseras-de-actividad/
Disallow: /electrodomesticos/ventiladores/de-techo-con-luz/
Disallow: /electrodomesticos/ventiladores/de-techo-con-luz/*
Disallow: /app-empleados/
Disallow: /app-empleados/*
Disallow: /promociones/empleados-eci/
Disallow: /promociones/empleados-eci/*
Disallow: /informacioncorporativa/es/comunicacion/notas-de-prensa/?*
Disallow: /informacioncorporativa/es/cargarMediaMediaElement.do* 


In [4]:
#Entrem a la pàgina de ElCorteInglés, secció Supermercado i obtenim les categories
categories = lookCategoria(path+"/supermercado",UserAg)
categories

Downloading: http://www.elcorteingles.es/supermercado


,link,nom
0,/supermercado/alimentacion-general/,Alimentación
1,/supermercado/desayunos-dulces-y-pan/,"Desayunos, Dulces y Pan"
2,/supermercado/lacteos/,Lácteos
3,/supermercado/congelados/,Congelados
4,/supermercado/dieteticos/,Dietética
5,/supermercado/bebidas/,Bebidas
6,/supermercado/frescos/,Frescos
7,/supermercado/bebes/,Bebés
8,/supermercado/higiene-personal/,Higiene
9,/supermercado/drogueria-y-limpieza/,Droguería


In [5]:
#Només buscarem productes de la secció de 'Congelados' per simplificar l'exercici (posició '3')
#Obtenim els tipus de productes dins la categoria 'Congelados'
Tipus = lookTipu(path+categories["link"][3],UserAg,categories["nom"][3])
Tipus

Downloading: http://www.elcorteingles.es/supermercado/congelados/


,categoria,link,nom
0,Congelados,/supermercado/congelados/aves-y-carne-congelada/,Aves y carne congelada
1,Congelados,/supermercado/congelados/helados-postres-y-nata/,"Helados, postres y nata"
2,Congelados,/supermercado/congelados/pan-y-reposteria-cong...,Pan y repostería congelada
3,Congelados,/supermercado/congelados/pescados-mariscos-y-s...,"Pescados, mariscos y surimis"
4,Congelados,/supermercado/congelados/pizza-lasana-y-canelo...,"Pizza, Lasaña y Canelones"
5,Congelados,/supermercado/congelados/platos-preparados-con...,Platos preparados congelados
6,Congelados,/supermercado/congelados/precocinados-rebozados/,Precocinados rebozados
7,Congelados,/supermercado/congelados/salteados-congelados/,Salteados congelados
8,Congelados,/supermercado/congelados/verduras-hortalizas-y...,"Verduras, hortalizas y frutas"


In [6]:
#Anem a buscar els productes de cada tipus de productes. Controlem la paginació que pugui haver
Prod = []

for x in range(len(Tipus['link'])):
    print(Tipus['nom'][x])
    Prod.append(lookProd(path+Tipus["link"][x],UserAg,Tipus["categoria"][x],Tipus["nom"][x]))
    if Prod[-1]['paginas'][0]>1:      
        for y in range(2,Prod[-1]['paginas'][0]+1):   #recorre les paginacions de productes del mateix tipus
            Prod.append(lookProdSeg(path+Tipus["link"][x]+str(y),UserAg,Tipus["categoria"][x],Tipus["nom"][x]))


Aves y carne congelada
Downloading: http://www.elcorteingles.es/supermercado/congelados/aves-y-carne-congelada/
Helados, postres y nata
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/2
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/3
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/4
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/5
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/6
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/7
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/8
Downloading: http://www.elcorteingles.es/supermercado/congelados/helados-postres-y-nata/9
Downloading: http://www.elcorteingles.es/supermercado/c

In [7]:
#Ara ja tenim la llista de prodcutes. Els consultarem un a un i n'extreurem els al·lèrgens i la imatge del producte.
#Creem a més un CSV amb la informació de tots els productes i una carpeta amb totes les imatges. 
#Degut a que hem detectat productes sense ingredients informats, els que no tinguin informació en els ingredients no 
#els inclourem a la llista.
Lista=[]
headerLista=["Categoria","Tipu","Producte","Imatge","Alergens"]
Lista.append(headerLista)


for t in range(len(Prod)):
    for x in range(len(Prod[t]["link"])):
        A = lookAlerg(path+Prod[t]["link"][x],UserAg,Prod[t]["nom"][x])
        if len(A)>0:
            valorLista=[Lista.append([Prod[t]["categoria"][x],
                                          Prod[t]["tipu"][x],
                                          Prod[t]["nom"][x],
                                          A[2].rsplit('/', 1)[-1],
                                          A[1]])]             #Només volem el nom del fitxer, no el path sencer
            img = load_requests("http:"+A[2],destiIMG)
       # if x==3: break     #descomentar si només volem fer una simulació amb menys dades.

with open(destiCSV+filename, 'w', newline='') as csvFile:
  writer = csv.writer(csvFile)
  for valor in Lista:
    writer.writerow(valor)


Downloading: http://www.elcorteingles.es/supermercado/0110118935100059-simons-hamburguesa-burkebab-estuche-240-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900083-simons-hamburguesa-de-buey-envase-200-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900141-simons-hamburguesa-especial-ternera-envase-200-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900158-simons-hamburguesa-retinto-estuche-200-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900273-brooklyn-town-burgers-hamburguesa-black-angus-usa-deluxe-100-carne-sin-aditivos-sin-gluten-talla-l-2-x130-g-estuche-260-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900257-brooklyn-town-hamburguesa-de-vacuno-de-raza-100-carne-sin-aditivos-clasica-sin-gluten-talla-l-2x130g-estuche-260-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118912900240-brooklyn-town-hamburguesa-de-ternera-ecologica-100-carne-sin-aditivos-sin-gluten-talla-l

Downloading: http://www.elcorteingles.es/supermercado/0110118952004838-magnum-pints-almond-helado-de-vainilla-con-trozos-de-almendra-y-cobertura-de-chocolate-con-leche-tarrina-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118952004820-magnum-pints-classic-helado-de-vainilla-con-trozos-de-chocolate-y-cobertura-de-chocolate-tarrina-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118952005306-magnum-pints-raspberry-helado-de-frambuesa-con-crujiente-capa-y-laminas-de-chocolate-negro-tarrina-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118952004846-magnum-pints-white-helado-de-vainilla-con-trozos-y-cobertura-de-chocolate-blanco-tarrina-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118952006015-magnum-praline-helado-de-crema-de-avellana-con-cobertura-de-chocolate-con-leche-tarrina-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118952006007-breyers-delight-helado-de-chocolate-bajo-en-calorias-tar

Downloading: http://www.elcorteingles.es/supermercado/0110118953906668-magnum-intense-dark-helado-de-chocolate-con-cobertura-de-chocolate-negro-sin-gluten-70-3-unids-estuche-300-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953907021-el-corte-ingles-bombon-helado-almendrado-4-unidades-sin-gluten-estuche-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953907047-el-corte-ingles-bombon-helado-de-chocolate-blanco-sin-gluten-4-unidades-estuche-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953907039-el-corte-ingles-bombon-helado-de-chocolate-negro-sin-gluten-4-unidades-estuche-440-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953907013-el-corte-ingles-minibombon-helado-sabor-a-vainilla-almendrado-sin-gluten-9-unidades-estuche-450-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953907054-el-corte-ingles-minibombon-helado-surtido-sin-gluten-9-unidades-estuche-450-ml/
Downloading: http://www.elcor

Downloading: http://www.elcorteingles.es/supermercado/0110118953907328-cornetto-go-sandwich-de-nata-y-trocitos-de-galleta-al-cacao-entre-galletas-de-chocolate-4-unidades-estuche-272-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953906460-el-corte-ingles-sandwich-de-helado-de-vainilla-y-chocolate-6-unidades-estuche-600-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953906478-el-corte-ingles-sandwich-de-helado-sabor-a-nata-6-unidades-estuche-600-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953906452-el-corte-ingles-sandwich-de-helado-sabor-a-nata-sin-azucares-anadidos-6-unidades-estuche-600-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953906494-el-corte-ingles-supersandwich-helado-de-stracciatella-con-cobertura-de-galleta-y-chocolate-4-unidades-estuche-600-ml/
Downloading: http://www.elcorteingles.es/supermercado/0110118953906825-el-corte-ingles-supersandwich-helado-de-vainilla-y-cobertura-de-chocolate-4-unidade

Downloading: http://www.elcorteingles.es/supermercado/0110118955801099-philadelphia-tarta-de-queso-cheesecake-ny-6-raciones-estuche-350-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118955801115-granderroble-carrot-cake-tarta-de-zanahoria-8-raciones-blister-770-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118955801081-oreo-tarta-con-galletas-cheesecake-6-raciones-estuche-350-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118934600034-nestle-miko-etiqueta-negra-tarta-al-whisky-con-crema-y-nata-10-raciones-estuche-1-l/
Downloading: http://www.elcorteingles.es/supermercado/0110118955800901-tentadore-tarta-selva-negra-sabor-chocolate-y-trufa-estuche-1100-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118955801412-eurochef-postre-cremoso-de-avellana-y-cafe-ecologico-sin-gluten-bandeja-110-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118955801453-eurochef-postre-cremoso-de-pistacho-y-frambuesa-ecologico-sin-gluten-ban

...

Downloading: http://www.elcorteingles.es/supermercado/0110118959601354-maheso-mussaka-lasana-de-berenjena-estuche-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958300784-maheso-quinoa-con-kale-y-lentejas-bolsa-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118931301727-fridela-pimientos-del-piquillo-rellenos-de-bacalao-2-unidades-estuche-120-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118925200604-vici-gyoza-dumplings-con-verduras-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118925200612-vici-gyoza-dumplings-con-verduras-picantes-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118925200455-vici-gyoza-empanadillas-de-verduras-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118931301180-la-cocina-de-senen-pintxo-de-huevo-frito-con-patatas-y-bacon-sin-gluten-bandeja-6-unidades/
Downloading: http://www.elcorteingles.es/supermercado/0110118931301818-la-cocina-de-s

Downloading: http://www.elcorteingles.es/supermercado/0110118949300638-pescanova-filetes-de-merluza-empanados-4-unidades-estuche-340-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300430-pescanova-chanquitos-enharinados-sin-gluten-y-sin-lactosa-bolsa-150-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300422-pescanova-palitos-rebozados-de-merluza-envase-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300026-pescanova-varitas-de-merluza-empanada-10-unidades-estuche-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300133-pescanova-varitas-de-merluza-rebozadas-con-queso-10-unidades-estuche-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300109-pescanova-lomos-a-la-romana-de-merluza-rebozada-5-unidades-estuche-325-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118949300398-pescanova-varitas-de-merluza-empanada-popcorn-al-horno-sin-gluten-10-unidades-estuche-300-g/
Downlo

Downloading: http://www.elcorteingles.es/supermercado/0110118924700794-maheso-croquetas-de-cocido-envase-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701339-la-ermita-de-san-pedro-croquetas-de-bacalao-bandeja-530-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701396-la-ermita-de-san-pedro-croquetas-de-carne-bandeja-530-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701362-la-ermita-de-san-pedro-croquetas-de-queso-do-picon-bejes-tresviso-bandeja-530-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701347-la-ermita-de-san-pedro-croquetas-de-jamon-iberico-bandeja-530-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701545-croquetas-ricas-gourmet-croquetas-de-bacalao-100-caseras-estuche-300-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118924701529-croquetas-ricas-gourmet-croquetas-de-boletus-edulis-100-caseras-estuche-300-g/
Downloading: http://www.elcorteingles.es/supermer

Downloading: http://www.elcorteingles.es/supermercado/0110118958400410-findus-salto-patata-con-jamon-y-queso-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958500193-findus-salto-noodles-con-verduras-y-pollo-plato-unico-1-racion-bolsa-350-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958200562-el-corte-ingles-arroz-tres-delicias-tradicional-bolsa-500-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958200547-el-corte-ingles-arroz-tres-delicias-con-pollo-bolsa-500-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958200554-el-corte-ingles-arroz-tres-delicias-con-marisco-bolsa-500-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958400477-el-corte-ingles-revuelto-de-gambas-ajetes-y-esparragos-trigueros-bolsa-450-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958300701-el-corte-ingles-salteado-de-verduras-asadas-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/011011

Downloading: http://www.elcorteingles.es/supermercado/0110118944500661-green-isle-patatas-para-horno-listas-para-asar-bolsa-800-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118944500653-felix-bolitas-de-patatas-bolsa-800-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800794-findus-calabacin-en-dados-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800570-findus-brocoli-estuche-675-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800745-findus-verduras-para-crema-y-pure-bolsa-750-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118958300651-findus-verduras-para-wok-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800620-findus-coliflor-bolsa-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800406-findus-grelos-hojas-2-porciones-estuche-400-g/
Downloading: http://www.elcorteingles.es/supermercado/0110118922800604-findus-acelgas-en-hojas-4-porcion